In [1]:
import sys
sys.path.append('/Users/ianleefmans/Desktop/stock-sentiment/utils')
from data import Database, ScrapeWSB, Stock
from datahelper import PostDataset, CommentDataset, get_indices
from torch.utils.data import DataLoader
from inference import RunInference
from models import FineTuneBaseModel, FineTuneClassifier
%cd /Users/ianleefmans/Desktop/stock-sentiment

/Users/ianleefmans/Desktop/stock-sentiment


In [2]:
import transformers
import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [4]:

db = Database()
db.use_database('DB1')

print(db.query('show tables;'))

connection established
[(datetime.datetime(2021, 10, 6, 18, 26, 27),)]
[('COMMENTS',), ('POSTS',), ('STOCKS',)]


In [7]:
db.query("SELECT CREATED FROM POSTS LIMIT 5")

[(1612120000.0,),
 (1612600000.0,),
 (1612830000.0,),
 (1613130000.0,),
 (1613620000.0,)]

In [7]:
db.query("SELECT * FROM STOCKS WHERE STOCK_ID='AMC' AND LAST_SCRAPED >= DATE_SUB(NOW(),INTERVAL 55 MINUTE)")

[('AMC', datetime.datetime(2021, 6, 6, 22, 15, 44))]

In [16]:
db.query("SELECT POST_ID FROM POSTS WHERE STOCK_ID='{}' AND LAST_SCRAPED >= DATE_SUB((SELECT LAST_SCRAPED FROM STOCKS WHERE STOCK_ID = '{}'), INTERVAL {} HOUR);".format('BB', 'BB', 6))



[('nx2ihx',),
 ('nxcb5h',),
 ('nxp5r6',),
 ('nxtjj4',),
 ('nxv9jr',),
 ('nxx9sz',),
 ('ny01wx',),
 ('nz6tgd',),
 ('nzf0mj',),
 ('nzjcfg',)]

In [86]:
db.query("SELECT POST_ID FROM POSTS WHERE STOCK_ID='{}' AND LAST_SCRAPED >= DATE_SUB((SELECT LAST_SCRAPED FROM STOCKS WHERE STOCK_ID = '{}'),INTERVAL {} HOUR);".format('AAPL','AAPL',6))



[('nv0h06',)]

In [85]:
db.query("SELECT LAST_SCRAPED FROM POSTS WHERE LAST_SCRAPED >= DATE_SUB((SELECT LAST_SCRAPED FROM STOCKS WHERE STOCK_ID = 'AAPL'),INTERVAL 6 HOUR);")
         
         
         


[(datetime.datetime(2021, 6, 14, 18, 21, 27),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 43),),
 (datetime.datetime(2021, 6, 14, 14, 27, 44),),
 (datetime.datetime(2021, 6, 14, 14, 27, 43),),
 (datetime.datetime(2021, 6, 14, 14, 27, 43),),
 (datetime.datetime(2021, 6, 14, 14, 27, 43),)]

In [89]:
db.query("SELECT POST_ID, LAST_SCRAPED FROM POSTS WHERE STOCK_ID='AAPL'")

[('mzzf2q', datetime.datetime(2021, 6, 7, 1, 38, 7)),
 ('n0ivmx', datetime.datetime(2021, 6, 7, 1, 38, 7)),
 ('n1kq5z', datetime.datetime(2021, 6, 7, 1, 38, 6)),
 ('n226xb', datetime.datetime(2021, 6, 7, 1, 38, 6)),
 ('n2guu3', datetime.datetime(2021, 6, 7, 1, 38, 6)),
 ('n3m6m0', datetime.datetime(2021, 6, 7, 1, 38, 5)),
 ('n51zx5', datetime.datetime(2021, 6, 7, 1, 38, 7)),
 ('n5sglk', datetime.datetime(2021, 6, 7, 1, 38, 7)),
 ('n6991f', datetime.datetime(2021, 6, 7, 1, 38, 7)),
 ('n6fa9f', datetime.datetime(2021, 6, 7, 1, 38, 5)),
 ('n6k7wf', datetime.datetime(2021, 6, 7, 1, 38, 6)),
 ('n6m3qj', datetime.datetime(2021, 6, 7, 1, 38, 5)),
 ('n799ne', datetime.datetime(2021, 6, 7, 1, 38, 6)),
 ('n7fe53', datetime.datetime(2021, 6, 7, 1, 38, 6)),
 ('n8coni', datetime.datetime(2021, 6, 7, 1, 38, 6)),
 ('n8mv7h', datetime.datetime(2021, 6, 7, 1, 38, 5)),
 ('n90iyg', datetime.datetime(2021, 6, 7, 1, 38, 5)),
 ('n93o6x', datetime.datetime(2021, 6, 7, 1, 38, 5)),
 ('na8f8j', datetime.datetim

In [45]:
indices = db.query("SELECT POST_ID FROM POSTS WHERE StOCK_ID='{}';".format("GME"))

In [4]:
db = Database()
db.use_database('DB1')
#db.drop_table("LABELED_POSTS")
#db.drop_table("LABELED_COMMENTS")

db.initialize_tables()

scrapewsb = ScrapeWSB('GME', 11, 2)

df = scrapewsb.scrape()
scrapewsb.convert(df)
print("DONE")

connection established
[(datetime.datetime(2021, 6, 6, 17, 23, 59),)]
connection established
[(datetime.datetime(2021, 6, 6, 17, 24),)]


  0%|          | 0/11 [00:00<?, ?it/s]

connection established
[(datetime.datetime(2021, 6, 6, 17, 24, 3),)]


100%|██████████| 11/11 [01:08<00:00,  6.24s/it]

DONE


In [4]:
db = Database()
db.use_database('DB1')

connection established
[(datetime.datetime(2021, 5, 30, 0, 20, 6),)]


In [3]:
db.drop_table('COMMENTS')
db.drop_table("POSTS")
db.drop_table("STOCKS")

In [2]:
indices = get_indices("GME", inference=True, scrape_time=6)

connection established
[(datetime.datetime(2021, 10, 6, 19, 48, 59),)]


In [3]:
post_train = PostDataset(512, indices['post_ids'])



connection established
[(datetime.datetime(2021, 10, 6, 19, 49, 1),)]


In [4]:
post_trainloader = DataLoader(
    dataset=post_train, 
    batch_size=3, 
    num_workers=0,
    shuffle=True
    )

In [5]:
sample = iter(post_trainloader).next()

In [6]:
sample


{'post': ['GIANT $RUN LOVING APE BALLS OUT.....$BB $AMC $GME',
  'Jim Cramer WSB FOMO- GME and AMC bull!',
  'Daily option statistics for AMC, CLNE, BB, WISH, GME, WKHS, CLOV, PLTR, CLF, and others.'],
 'post_input_ids': tensor([[ 101,  144, 9984,  ...,    0,    0,    0],
         [ 101, 3104,  140,  ...,    0,    0,    0],
         [ 101, 5732, 5146,  ...,    0,    0,    0]]),
 'post_attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'target': tensor([-1, -1, -1]),
 'date': tensor([1.6240e+09, 1.6240e+09, 1.6240e+09], dtype=torch.float64)}

In [7]:
model = FineTuneClassifier()

out = model(input_ids=sample['post_input_ids'], attention_masks=sample['post_attention_mask'])

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [8]:
out

tensor([[0.5016, 0.4984],
        [0.5107, 0.4893],
        [0.4868, 0.5132]], grad_fn=<SoftmaxBackward>)

In [20]:
max_pos = {}

In [18]:
float(out[:,1].max())

0.5132220983505249

In [21]:
max_pos[float(out[:,1].max())] = sample['post'][int(out[:,1].argmax())]

In [22]:
max_pos

{0.5132220983505249: 'AMC YOLO LFG!'}

In [2]:
run_inference = RunInference(stock_id='GME')
inference_output = run_inference.evaluate()

post_probs = inference_output['avg_post_probs']

comment_probs = inference_output['avg_comment_probs']

connection established
[(datetime.datetime(2021, 6, 18, 0, 20, 42),)]
connection established
[(datetime.datetime(2021, 6, 18, 0, 20, 44),)]


Determining Sentiment From Posts:   0%|          | 0/1 [00:00<?, ?it/s]

connection established
[(datetime.datetime(2021, 6, 18, 0, 20, 45),)]


Determining Sentiment From Comments: 100%|██████████| 1/1 [00:06<00:00,  6.85s/it]


In [3]:
all_post_probs = inference_output['all_post_probs']
all_comment_probs = inference_output['all_comment_probs']

In [4]:
all_post_probs

tensor([[9.9838e-01, 1.6161e-03],
        [3.9883e-04, 9.9960e-01],
        [1.4419e-04, 9.9986e-01]])

In [5]:
all_comment_probs

tensor([[0.9948, 0.0052],
        [0.0657, 0.9343],
        [0.9922, 0.0078]])

In [2]:
stock = Stock()
stock.set_start([2021, 7, 16, 0, 0, 0])
data = stock.pull_data('GME')

100%|██████████| 6476/6476 [00:20<00:00, 323.22it/s]


In [8]:
len(data)

4876

In [7]:
data.dropna(axis=0, inplace=True)

In [4]:
data['highlow_percent'] = data.low/data.high


In [3]:
def lag(feature, df):
    df[feature+"_t1"] = [df.loc[df.timestamp.dt.day==i.day-1,['timestamp', feature]]['feature'].mean() for i in tqdm(df.timestamp)]






In [5]:
data

,close,high,low,open,status,timestamp,volume,highlow_percent,target
0,172.99,172.99,172.99,172.99,ok,2021-07-16 04:00:00,170,1.000000,NaN
1,173.10,173.10,173.00,173.00,ok,2021-07-16 04:03:00,385,0.999422,NaN
2,174.00,174.00,174.00,174.00,ok,2021-07-16 04:04:00,130,1.000000,NaN
3,174.01,174.01,174.01,174.01,ok,2021-07-16 04:05:00,359,1.000000,NaN
4,174.00,174.00,174.00,174.00,ok,2021-07-16 04:14:00,457,1.000000,NaN
...,...,...,...,...,...,...,...,...,...
6471,151.95,151.95,151.95,151.95,ok,2021-08-06 17:06:00,290,1.000000,152.645524
6472,152.00,152.00,152.00,152.00,ok,2021-08-06 17:30:00,865,1.000000,152.645524
6473,152.00,152.00,152.00,152.00,ok,2021-08-06 18:10:00,131,1.000000,152.645524
6474,152.01,152.01,152.01,152.01,ok,2021-08-06 18:35:00,202,1.000000,152.645524


In [8]:
import pandas as pd
data.timestamp = pd.to_datetime(data.timestamp)

In [4]:

from statsmodels.tsa.arima.model import ARIMA

In [9]:
mod = ARIMA(endog=data.close, exog = data.highlow_percent, order=(1, 0, 0))
res = mod.fit()
print(res.summary())

                               SARIMAX Results                                
Dep. Variable:                  close   No. Observations:                 6476
Model:                 ARIMA(1, 0, 0)   Log Likelihood               -3784.048
Date:                Sun, 08 Aug 2021   AIC                           7576.096
Time:                        16:40:59   BIC                           7603.199
Sample:                             0   HQIC                          7585.472
                               - 6476                                         
Covariance Type:                  opg                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              64.2753    108.867      0.590      0.555    -149.101     277.651
highlow_percent     7.0061      0.943      7.426      0.000       5.157       8.855
ar.L1               1.0000   5.8

In [37]:
from tqdm import tqdm

data['target'] = [data.loc[data.timestamp.dt.day==i.day-1,['timestamp', 'close']].close.mean() for i in tqdm(data.timestamp)]




100%|██████████| 6476/6476 [00:21<00:00, 303.90it/s]


In [34]:
data.loc[data.timestamp.dt.day==16,['timestamp', 'close']].close.mean()

172.5266180555556

In [33]:
from datetime import datetime, timedelta


In [15]:
day = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day[datetime.today().weekday()]

'Tuesday'

In [17]:
now = datetime.today()

In [21]:
now.

20

In [22]:
now

datetime.datetime(2021, 7, 20, 20, 29, 21, 976621)

In [25]:
now - timedelta(days=3)

datetime.datetime(2021, 7, 17, 20, 29, 21, 976621)

In [2]:
from inference import RunInference

run =RunInference('GME')
output = run.evaluate()

connection established
[(datetime.datetime(2021, 10, 6, 20, 23, 51),)]
connection established
[(datetime.datetime(2021, 10, 6, 20, 23, 53),)]


Determining Sentiment From Posts:   0%|          | 0/1 [00:00<?, ?it/s]

connection established
[(datetime.datetime(2021, 10, 6, 20, 23, 54),)]


Determining Sentiment From Comments: 100%|██████████| 1/1 [00:09<00:00,  9.48s/it]


In [3]:
output

{'avg_post_probs': tensor([3.6181e-04, 9.9964e-01]),
 'avg_comment_probs': tensor([0.0803, 0.9197]),
 'all_post_probs': tensor([[6.6488e-04, 9.9934e-01],
         [1.8537e-04, 9.9981e-01],
         [5.7578e-04, 9.9942e-01],
         [2.1131e-04, 9.9979e-01],
         [1.7168e-04, 9.9983e-01]]),
 'all_comment_probs': tensor([[4.0072e-01, 5.9928e-01],
         [1.6010e-04, 9.9984e-01],
         [3.3146e-04, 9.9967e-01],
         [1.6648e-04, 9.9983e-01],
         [1.9659e-04, 9.9980e-01]]),
 'all_post_dates': tensor([1.6240e+09, 1.6240e+09, 1.6239e+09, 1.6239e+09, 1.6240e+09],
        dtype=torch.float64),
 'all_comment_dates': tensor([1.6240e+09, 1.6239e+09, 1.6240e+09, 1.6240e+09, 1.6239e+09],
        dtype=torch.float64),
 'max_post_prob': 0.9998283386230469,
 'max_comment_prob': 0.9998399019241333,
 'max_post': 'Jim Cramer WSB FOMO- GME and AMC bull!',
 'max_comment': '#BB GANG 🦍🦍🦍',
 'min_post_prob': 0.9993351101875305,
 'min_comment_prob': 0.5992765426635742,
 'min_post': 'Daily op

In [8]:
from datetime import datetime
print(datetime.utcfromtimestamp(output['all_post_dates'][0]).strftime('%Y-%m-%d %H:%M:%S'))

2021-06-17 03:20:00


In [14]:
import pandas as pd

get_prob = lambda x: float(x[1])
get_date = lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')

df1 = pd.DataFrame({'date': list(map(get_date, output['all_post_dates'])), 
              'sentiment': list(map(get_prob, output['all_post_probs']))})

In [15]:
df2 = pd.DataFrame({'date': list(map(get_date, output['all_comment_dates'])), 
              'sentiment': list(map(get_prob, output['all_comment_probs']))})

In [18]:
df1

,date,sentiment
0,2021-06-17 03:20:00,0.999424
1,2021-06-17 06:06:40,0.999789
2,2021-06-17 22:46:40,0.999828
3,2021-06-18 04:20:00,0.999335
4,2021-06-18 07:06:40,0.999815


In [19]:
df2

,date,sentiment
0,2021-06-18 04:20:00,0.999669
1,2021-06-17 03:20:00,0.999803
2,2021-06-18 07:06:40,0.599277
3,2021-06-17 22:46:40,0.999833
4,2021-06-17 06:06:40,0.999840


In [17]:
pd.concat([df1, df2])

,date,sentiment
0,2021-06-17 03:20:00,0.999424
1,2021-06-17 06:06:40,0.999789
2,2021-06-17 22:46:40,0.999828
3,2021-06-18 04:20:00,0.999335
4,2021-06-18 07:06:40,0.999815
0,2021-06-18 04:20:00,0.999669
1,2021-06-17 03:20:00,0.999803
2,2021-06-18 07:06:40,0.599277
3,2021-06-17 22:46:40,0.999833
4,2021-06-17 06:06:40,0.999840


In [4]:
from forecast import Forecast

get_stock = Forecast("GME", output)

100%|██████████| 1081/1081 [00:02<00:00, 433.85it/s]


In [5]:
get_stock.sentiment

,date,sentiment
0,2021-06-18 04:20:00,0.999335
1,2021-06-18 07:06:40,0.999815
2,2021-06-17 03:20:00,0.999424
3,2021-06-17 06:06:40,0.999789
4,2021-06-17 22:46:40,0.999828
0,2021-06-18 07:06:40,0.599277
1,2021-06-17 06:06:40,0.999840
2,2021-06-18 04:20:00,0.999669
3,2021-06-17 22:46:40,0.999833
4,2021-06-17 03:20:00,0.999803


In [5]:
data = get_stock.stock_data

In [6]:
data

,close,high,low,open,status,timestamp,volume,highlow_percent,highlow_percent_t1
0,176.330,176.340,176.330,176.340,ok,2021-10-01 13:39:00,1677,0.999943,NaN
1,176.150,176.220,176.150,176.220,ok,2021-10-01 13:40:00,915,0.999603,NaN
2,176.030,176.030,176.030,176.030,ok,2021-10-01 13:41:00,386,1.000000,NaN
3,175.970,176.110,175.970,176.000,ok,2021-10-01 13:42:00,2029,0.999205,NaN
4,176.110,176.110,176.110,176.110,ok,2021-10-01 13:43:00,803,1.000000,NaN
...,...,...,...,...,...,...,...,...,...
1085,169.705,169.705,169.705,169.705,ok,2021-10-06 13:33:00,1284,1.000000,0.99874
1086,169.870,170.120,169.780,169.780,ok,2021-10-06 13:34:00,4059,0.998001,0.99874
1087,170.100,170.190,170.030,170.190,ok,2021-10-06 13:35:00,1953,0.999060,0.99874
1088,170.050,170.125,169.880,170.125,ok,2021-10-06 13:36:00,2156,0.998560,0.99874


In [7]:
data.dropna(axis=0, inplace=True)

In [8]:
data

,close,high,low,open,status,timestamp,volume,highlow_percent,highlow_percent_t1
471,171.360,171.360,171.360,171.360,ok,2021-10-05 04:07:00,193,1.000000,0.998716
472,172.160,172.160,172.150,172.150,ok,2021-10-05 07:33:00,383,0.999942,0.998716
473,172.000,172.050,172.000,172.050,ok,2021-10-05 08:10:00,416,0.999709,0.998716
474,171.810,171.810,171.810,171.810,ok,2021-10-05 08:11:00,468,1.000000,0.998716
475,171.020,171.020,171.020,171.020,ok,2021-10-05 08:12:00,525,1.000000,0.998716
...,...,...,...,...,...,...,...,...,...
1085,169.705,169.705,169.705,169.705,ok,2021-10-06 13:33:00,1284,1.000000,0.998740
1086,169.870,170.120,169.780,169.780,ok,2021-10-06 13:34:00,4059,0.998001,0.998740
1087,170.100,170.190,170.030,170.190,ok,2021-10-06 13:35:00,1953,0.999060,0.998740
1088,170.050,170.125,169.880,170.125,ok,2021-10-06 13:36:00,2156,0.998560,0.998740


In [4]:
results = get_stock.arima()

In [6]:
print(results.summary())

                               SARIMAX Results                                
Dep. Variable:                  close   No. Observations:                 1090
Model:                 ARIMA(1, 0, 0)   Log Likelihood                -406.430
Date:                Wed, 06 Oct 2021   AIC                            820.860
Time:                        13:56:50   BIC                            840.836
Sample:                             0   HQIC                           828.421
                               - 1090                                         
Covariance Type:                  opg                                         
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             139.7038      0.002   5.63e+04      0.000     139.699     139.709
highlow_percent     3.9948      2.846      1.403      0.160      -1.584       9.574
ar.L1               1.0000   1.3

In [ ]:
results.predict 